# Activity 6. Bearing Defect Type Classification

### **2019/7/1 SK Hynix - KAIST**<br/>
<br/>

***Tip> shotcuts for Jupyter Notebook***
* Shift + Enter : run cell and select below

# 0. Introduction 

<font size="3">베어링에는 크게 볼 손상, 내륜 손상, 외륜 손상의 3가지 결함이 있습니다.<br/>
이번 시간에는 Deep Neural Network를 사용하여, 베어링의 정상 신호와 3가지 결함을 분류해보는 문제를 풀어보도록 하겠습니다.
</font>

![title](img\bearing.PNG)

![title](img\defect.PNG)

# 1. Module

<font size="3">
실습에 필요한 module을 불러옵니다. <br/>

Tensorflow : Deep learning 학습에 최적화된 module <br/>
numpy : 수학적 연산에 관련된 module <br/>
os : system 내부 제어에 사용되는 module
</font>

In [ ]:
import tensorflow as tf
import numpy as np
import os

# 2. Data

<font size="3">
실습에 필요한 data를 불러오겠습니다. <br/>
외륜 손상은 'OR', 볼 손상은 'B', 내륜 손상은 'IR', 정상은 'N'으로 나타내겠습니다. <br/>

</font>

## 2-(1). Data Loading

<font size="3">
베어링 신호가 위치한 폴더에 접근하고, 코드 구현의 편의성을 위해 각 손상 class별로 분류해줍니다.
</font>

In [ ]:
# 폴더 내 파일 목록 불러오기
file_list = os.listdir('./CWRU_360_npy')

# 분류를 위한 empty list들 생성
B_files, IR_files, OR_files, N_files = [], [], [], []

# 손상 class별 파일 분류
for item in file_list:
    if item.find('B') is not -1 : # 해당 문자열을 이름에 포함하지 않는 경우 -1을 return함.
        B_files.append(item)
    if item.find('IR') is not -1 : 
        IR_files.append(item)
    if item.find('OR') is not -1 : 
        OR_files.append(item)
    if item.find('N') is not -1 : 
        N_files.append(item)

<font size="3">
각 class 별 파일의 개수를 확인해보겠습니다.
</font>

In [ ]:
print('Defect type 별 파일의 수 : 볼손상 - %d, 내륜손상 - %d, 외륜손상 - %d, 정상 - %d' 
      %(len(B_files), len(IR_files), len(OR_files), len(N_files)))

## 2-(2). Pre-processing

<font size="3">
각 class에 해당하는 label은 다음과 같이 one-hot으로 encoding 하겠습니다. <br/>
볼 손상 : [1,0,0,0] / 내륜 손상 : [0,1,0,0] / 외륜 손상 : [0,0,1,0] / 정상 : [0,0,0,1] <br/>
</font>

In [ ]:
# label
label = {'B': [1,0,0,0], 'IR' : [0,1,0,0], 'OR' : [0,0,1,0], 'N' : [0,0,0,1]}

<font size="3">
볼, 내륜, 외륜 손상은 파일 별로 200개의 vector가 생성되며, 정상 데이터는 파일 별로 600개의 vector가 생성됩니다. 
</font>

In [ ]:
# the number of vectors per file
B_vector_num = 200
IR_vector_num = 200
OR_vector_num = 200
N_vector_num = 600

<font size="3">
각 data 파일 별로 min-max scaling을 적용하고, 360 points 씩 vector를 구성하도록 하겠습니다. <br/>
</font>

In [ ]:
## 볼 손상 vector generation ##

B = []
B_label = []

for i in range(len(B_files)):
    B_data = np.load('./CWRU_360_npy/' + B_files[i])
    
    # min-max scaling
    B_data_scaling = B_data / (np.max(B_data) - np.min(B_data)) 

    
    for j in range(0, B_vector_num):
        B_tmp = B_data_scaling[j*360 : (j+1)*360]
        B.append(B_tmp.tolist())
        B_label.append(label['B'])
        B_tmp = []

In [ ]:
## 내륜 손상 vector generation ##

IR = []
IR_label = []

for i in range(len(IR_files)):
    IR_data = np.load('./CWRU_360_npy/' + IR_files[i])
    
    # min-max scaling
    IR_data_scaling = IR_data / (np.max(IR_data) - np.min(IR_data)) 
    
    
    for j in range(0, IR_vector_num):
        IR_tmp = IR_data_scaling[j*360 : (j+1)*360]
        IR.append(IR_tmp.tolist())
        IR_label.append(label['IR'])
        IR_tmp = []

In [ ]:
## 외륜 손상 vector generation ##

OR = []
OR_label = []

for i in range(len(OR_files)):
    OR_data = np.load('./CWRU_360_npy/' + OR_files[i])
    
    # min-max scaling
    OR_data_scaling = OR_data / (np.max(OR_data) - np.min(OR_data)) 
    
    
    for j in range(0, OR_vector_num):
        OR_tmp = OR_data_scaling[j*360 : (j+1)*360]
        OR.append(OR_tmp.tolist())
        OR_label.append(label['OR'])
        OR_tmp = []

In [ ]:
## 정상 vector generation ##

N = []
N_label = []

for i in range(len(N_files)):
    N_data = np.load('./CWRU_360_npy/' + N_files[i])
    
    # min-max scaling
    N_data_scaling = N_data / (np.max(N_data) - np.min(N_data)) 
    

    
    for j in range(0, N_vector_num):
        N_tmp = N_data_scaling[j*360 : (j+1)*360]
        N.append(N_tmp.tolist())
        N_label.append(label['N'])
        N_tmp = []

<font size="3">
pre-processing이 끝난 data들을 구현상의 편의를 위해 자료형을 변환해주겠습니다.
</font>

In [ ]:
B_data = np.asarray(B)
B_label = np.asarray(B_label)

IR_data = np.asarray(IR)
IR_label = np.asarray(IR_label)

OR_data = np.asarray(OR)
OR_label = np.asarray(OR_label)

N_data = np.asarray(N)
N_label = np.asarray(N_label)

<font size="3">

총 4개 class의 데이터들을 360 points씩 분할하여 구성한 최종 dataset은 다음과 같습니다.

</font>

![title](img\dataset.PNG)

<font size="3">
각 결함의 파형을 확인해보겠습니다.(아래 cell을 2번 실행해주셔야 그림이 보입니다.)

</font>

In [ ]:
import matplotlib.pyplot as plt

fs=12000
t_tmp = np.arange(len(B_data[0]))
t = t_tmp/fs

plt.figure(figsize=(20,5))
plt.subplots_adjust(hspace = 0.5, wspace = 0.3)

plt.subplot(2,2,1)
plt.plot(t,B_data[0])
plt.ylim(-0.5, 0.5)
plt.title('Defect_B', fontsize=20)

plt.subplot(2,2,2)
plt.plot(t,IR_data[0])
plt.ylim(-0.5, 0.5)
plt.title('Defect_IR', fontsize=20)

plt.subplot(2,2,3)
plt.plot(t,OR_data[0])
plt.ylim(-0.5, 0.5)
plt.title('Defect_OR', fontsize=20)

plt.subplot(2,2,4)
plt.plot(t,N_data[0])
plt.ylim(-0.4, 0.4)
plt.title('Normal', fontsize=20)

plt.show()

## 2-(3). Train / Validation / Test split

<font size="3">
Model을 학습하기 위한 training set, model의 최적의 parameter를 결정하기 위한 validation set, model의 최종 성능을 평가할 test set을 구성하도록 하겠습니다.<br/>
Class별 balance를 유지하기 위해, 각 class별로 training:test = 8:2의 비율로 구성하고, training set에서 10%는 validation set으로 구성하겠습니다.
</font>

In [ ]:
from sklearn.model_selection import train_test_split
B_tmp, B_test, B_label_tmp, B_label_test = train_test_split(B_data, B_label, test_size = 0.2, random_state=42)
IR_tmp, IR_test, IR_label_tmp, IR_label_test = train_test_split(IR_data, IR_label, test_size = 0.2, random_state=42)
OR_tmp, OR_test, OR_label_tmp, OR_label_test = train_test_split(OR_data, OR_label, test_size = 0.2, random_state=42)
N_tmp, N_test, N_label_tmp, N_label_test = train_test_split(N_data, N_label, test_size = 0.2, random_state=42)

B_train, B_val, B_label_train, B_label_val = train_test_split(B_tmp, B_label_tmp, test_size = 0.1, random_state=42)
IR_train, IR_val, IR_label_train, IR_label_val = train_test_split(IR_tmp, IR_label_tmp, test_size = 0.1, random_state=42)
OR_train, OR_val, OR_label_train, OR_label_val = train_test_split(OR_tmp, OR_label_tmp, test_size = 0.1, random_state=42)
N_train, N_val, N_label_train, N_label_val = train_test_split(N_tmp, N_label_tmp, test_size = 0.1, random_state=42)

<font size="3">
각 Class별로 구성된 training, validation, test data들을 편하게 다루기 위하여 통합하겠습니다.
</font>

In [ ]:
train_data = np.concatenate((B_train, IR_train, OR_train, N_train), axis=0)
train_label = np.concatenate((B_label_train, IR_label_train, OR_label_train, N_label_train), axis=0)

val_data = np.concatenate((B_val, IR_val, OR_val, N_val), axis=0)
val_label = np.concatenate((B_label_val, IR_label_val, OR_label_val, N_label_val), axis=0)

test_data = np.concatenate((B_test, IR_test, OR_test, N_test), axis=0)
test_label = np.concatenate((B_label_test, IR_label_test, OR_label_test, N_label_test), axis=0)

# 3. Model

<font size="3">
Classification을 수행할 model을 생성하도록 하겠습니다.<br/>
아래 코드는 tensor들의 통로 역할을 default graph를 초기화하는 코드로써, 여러분의 실습을 원활하게 진행하기 위한 것이므로, 추후 네트워크 구조나 hyper parameter 수정시마다 실행을 해주셔야 합니다.

</font>

In [ ]:
tf.reset_default_graph()

## 3-(1) Hyper parameter setting

<font size="3">
Model training에 적용할 hyper parameter들을 설정합니다.
    
</font>

In [ ]:
n_epochs = 350
learning_rate = 0.00001
batch_size = 8

## 3-(2) Placeholder

<font size="3">
Model에 대한 input과 그에 대응하는 label을 담을 변수를 설정합니다. <br/>
</font>

In [ ]:
DNN_input = tf.placeholder(tf.float32, [None, 360], name='DNN_input') 
DNN_label = tf.placeholder(tf.float32, [None,4], name='DNN_label')

## 3-(3) DNN model(<span style="color:red">Fill in the blanks</span>)

<font size="3">
학습을 진행할 model을 설계합니다. <br/>
여기서는 layer별로 각각 360-150-50-4 의 neurons를 갖는 deep neural network를 사용해보겠습니다. <br/>
첫 hidden layer에 대한 code를 참고하여, 두번째 hidden layer와 마지막 output layer를 설계해 보세요. <br/>
(공통된 결과를 출력하기 위하여, initializer의 seed는 26으로 유지해주세요)
</font>

In [ ]:
def DNN(DNN_input):
    
   
    ## 1st hidden layer
    w1 = tf.get_variable(name='DNN_weight1', shape=[360, 150], initializer= tf.contrib.layers.xavier_initializer(seed=26))     
    b1 = tf.get_variable(name='DNN_biases1', shape=[150], initializer= tf.contrib.layers.xavier_initializer(seed=26))          
    h1 = tf.matmul(DNN_input, w1) + b1          
    h1_relu = tf.nn.relu(h1, name='DNN_h1_relu')                


    ## 2nd hidden layer
    w2 = 
    b2 = 
    h2 = 
    h2_relu = 
    
    
    ## output layer
    w3 = 
    b3 = 
    h3 = 
    h3_relu = 
    

    return h3_relu

## 3-(4) Cost, Optimizer

<font size="3">
Model의 학습을 진행할 optimizer, 성능을 평가할 cost와 accuracy를 설정합니다.
</font>

In [ ]:
DNN_out = DNN(DNN_input)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=DNN_label, logits=DNN_out), name='cost')

opt = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(DNN_out, 1), tf.argmax(DNN_label, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# 4. Model training

## 4-(1) Session

<font size="3">
Model을 실행할 session을 열고, 변수들을 초기화합니다.
</font>

In [ ]:
# saver
DNN_saver = tf.train.Saver(max_to_keep=1)

## MAKE SESSION
DNN_sess = tf.Session()

## INITIALIZE SESSION
DNN_sess.run(tf.global_variables_initializer())

## 4-(2) Training

<font size="3">
Model을 학습하고, 매 epoch마다 validation set을 통해 model을 평가하여, 최적의 parameter들을 찾아냅니다.
    
</font>

In [ ]:
last_best = 0.000000001

for epoch in range(n_epochs+1):
    Z = np.random.permutation(len(train_data))
    X_train_shuffle = train_data[Z]
    Y_train_shuffle = train_label[Z]
        
    for iteration in range(len(train_data) // batch_size):
        
        X_batch = X_train_shuffle[iteration*batch_size : (1+iteration)*batch_size]
        Y_batch = Y_train_shuffle[iteration*batch_size : (1+iteration)*batch_size]
        X_batch = X_batch.reshape([-1,360])
        Y_batch = Y_batch.reshape([-1,4])
        
        (_, training_cost) = DNN_sess.run([opt, cost], feed_dict={DNN_input: X_batch, DNN_label: Y_batch})

    
    ## Training accuracy every one epoch
    acc_train = accuracy.eval(session=DNN_sess, feed_dict={DNN_input: X_batch, DNN_label: Y_batch})
    if epoch % 1 == 0:
        print('  [*] TRAINING Iteration %d, Loss: %.6f' % (epoch, training_cost))

    ## Validation accuracy every 1 epochs
    if epoch % 1 == 0:
        acc_val = accuracy.eval(session=DNN_sess, feed_dict={DNN_input: val_data, DNN_label: val_label})
        print('  [*] VALIDATION ACC: %.4f' % acc_val)
    
    if acc_val > last_best:
        last_best = acc_val
        print('*******************************')
        DNN_saver.save(DNN_sess, './DNN.ckpt', global_step=epoch)
        best_epoch = epoch

print('Optimization done.')

In [ ]:
print('Optimal epoches : %d' %best_epoch)

In [ ]:
print('Optimal accuracy : %f' %last_best)

# 5. Model test

<font size="3">
최적의 model에 대하여, test set을 통해 model의 최종 성능을 평가합니다.
</font>

In [ ]:
DNN_saver.restore(DNN_sess, './DNN.ckpt' + '-' + str(best_epoch))

In [ ]:
accuracy.eval(session=DNN_sess, feed_dict={DNN_input: test_data, DNN_label: test_label})

In [ ]:
DNN_sess.close()
tf.reset_default_graph()

# 6. Quiz

### Q1. 3-(3)의 code를 완성하세요.

### Q2. 3-(3)에 완성된 network의 총 parameter 수를 계산하세요.

### Q3. 3-(1)과 3-(3)의 hyper parameter들과 network 구조를 변화시켜서, accuracy를 향상시켜 보세요.